## 导包

In [1]:
# 公榜分数0.92860690761
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import StratifiedKFold, KFold
from matplotlib.pyplot import plot, show

In [2]:
import os
import pandas as pd 
import numpy as np
import scorpyo as sp

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import time

from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from lightgbm import LGBMClassifier

In [3]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


## results
path_output_report = os.path.join(path_results, '01_原始数据探察.xlsx')

y_label = "is_risk"

## 数据读取

In [4]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

In [5]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,guojianping9672,rd,GVhZtW4i1,rqRxAjAL1RYC,firefox_78,firefox,win,win10,2022-01-18 19:10:41,内网,200,成都,2umVQwhiiwNJ,xxx.com/mail,2022-01,0
1,1,yangtao1740,sales,l3MuTMPoQ,iKPTa3su50y7,chrome_93,chrome,win,win11,2022-04-01 17:04:00,内网,200,深圳,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0
2,2,wangying9098,rd,4uHWcskWv,1baNbqxMWcCu,ie_11,ie,win,win10,2022-03-01 15:53:49,内网,200,成都,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-03,0
3,3,liguixiang3860,rd,mQh3NwtY7,C04Llg4lKl4C,edge_93,edge,win,win10,2022-02-07 19:46:25,内网,200,北京,nHrKgKdJ1Mzt,xxx.com/github,2022-02,0
4,4,guanyu9205,sales,C2QtgDKAZ,kSscjiRSz1aD,edge_93,edge,win,win10,2022-04-12 10:05:19,内网,200,成都,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0


In [6]:
_ = sp.excel_label(df_row_train, y=y_label, excel_path=path_output_report, show=True)

------------------------------------------------------------------------------------------
标签分布展示如下:

数据集样本有: 47660个,特征列有:16列, 标签列为: is_risk


,label取值,label数量,label占比
0,0,39964,83.85%
1,1,7696,16.15%


,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,guojianping9672,rd,GVhZtW4i1,rqRxAjAL1RYC,firefox_78,firefox,win,win10,2022-01-18 19:10:41,内网,200,成都,2umVQwhiiwNJ,xxx.com/mail,2022-01,0
1,1,yangtao1740,sales,l3MuTMPoQ,iKPTa3su50y7,chrome_93,chrome,win,win11,2022-04-01 17:04:00,内网,200,深圳,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0
2,2,wangying9098,rd,4uHWcskWv,1baNbqxMWcCu,ie_11,ie,win,win10,2022-03-01 15:53:49,内网,200,成都,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-03,0


sheet01.标签分布分析在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察.xlsx中已经存在，我们将对原文件进行覆盖


------------------------------------------------------------------------------------------


In [7]:
sp.excel_detect(df_row_train, sheet_name='02.训练集-数据描述性统计',excel_path=path_output_report)

sheet02.训练集-数据描述性统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察.xlsx中已经存在，我们将对原文件进行覆盖


,feat_name_row,type,size,missing,unique,zero_ratio,negative_ratio,top1_all_value,top1_all_ratio,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
0,id,int64,47660,0.00,47660,0.000,0.0,0,0.000,23829.5,13758.401,0.0,476.59,4765.9,23829.5,35744.25,42893.1,47182.41,47659.0
1,user_name,object,47660,0.08,187,0.000,NaN,xuxiuying8050,0.007,xuxiuying8050:0.65%,hongchang3029:0.63%,tanliu3173:0.62%,liuhong6350:0.62%,lufan2545:0.62%,zhouxiumei4433:0.38%,chenjian4844:0.37%,wanggang1192:0.36%,ranxiuzhen6780:0.33%,xujie9775:0.30%
2,department,object,47660,0.08,5,0.000,NaN,rd,0.654,rd:65.36%,sales:17.26%,other:4.07%,accounting:3.56%,hr:1.75%,rd:65.36%,sales:17.26%,other:4.07%,accounting:3.56%,hr:1.75%
3,ip_transform,object,47660,0.00,2105,0.000,NaN,w2CfuqTz3,0.007,w2CfuqTz3:0.68%,u9diCFdYZ:0.66%,pPgzIf3S4:0.65%,7YnPN3fqd:0.65%,DhTMwbtS5:0.64%,948U9MQcB:0.00%,h75YAkAAL:0.00%,m7512MutA:0.00%,ADL8GwW32:0.00%,g3dWezpzT:0.00%
4,device_num_transform,object,47660,0.00,844,0.000,NaN,O54DfqjlCrhL,0.007,O54DfqjlCrhL:0.70%,kUa61ygA6gI3:0.68%,Rfv57YyO3vny:0.67%,5DmlITfRNR36:0.66%,TzmgdvYq3Kx0:0.66%,aUECyyFo55Zy:0.00%,cREgOG9x3d9X:0.00%,NGfeE42d1yHY:0.00%,T4hueKNccs7X:0.00%,A0TLDctT8OUR:0.00%
5,browser_version,object,47660,0.00,8,0.000,NaN,edge_93,0.372,edge_93:37.16%,chrome_90:33.38%,safari_13:9.48%,chrome_77:5.32%,firefox_78:4.82%,chrome_77:5.32%,firefox_78:4.82%,chrome_93:4.44%,ie_11:3.37%,ie_9:2.02%
6,browser,object,47660,0.00,5,0.000,NaN,chrome,0.431,chrome:43.15%,edge:37.16%,safari:9.48%,ie:5.39%,firefox:4.82%,chrome:43.15%,edge:37.16%,safari:9.48%,ie:5.39%,firefox:4.82%
7,os_type,object,47660,0.00,2,0.000,NaN,win,0.905,win:90.52%,macos:9.48%,None,None,None,None,None,None,win:90.52%,macos:9.48%
8,os_version,object,47660,0.00,4,0.000,NaN,win10,0.758,win10:75.84%,win7:10.81%,macos_big_sur_11:9.48%,win11:3.87%,None,None,win10:75.84%,win7:10.81%,macos_big_sur_11:9.48%,win11:3.87%
9,op_datetime,object,47660,0.00,47343,0.000,NaN,2022-04-02 17:02:24,0.000,2022-04-02 17:02:24:0.01%,2022-02-16 11:36:12:0.01%,2022-04-28 09:33:53:0.01%,2022-03-04 17:13:36:0.01%,2022-03-23 14:47:36:0.00%,2022-04-29 09:58:59:0.00%,2022-04-20 19:02:48:0.00%,2022-01-19 15:11:24:0.00%,2022-01-19 10:53:46:0.00%,2022-04-08 19:30:10:0.00%


In [8]:
sp.excel_detect(df_row_val, sheet_name='03.测试集-数据描述性统计',excel_path=path_output_report)

sheet03.测试集-数据描述性统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察.xlsx中已经存在，我们将对原文件进行覆盖


,feat_name_row,type,size,missing,unique,zero_ratio,negative_ratio,top1_all_value,top1_all_ratio,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
0,id,int64,25710,0.000,25710,0.0,0.0,0,0.000,12854.5,7421.982,0.0,257.09,2570.9,12854.5,19281.75,23138.1,25451.91,25709.0
1,user_name,object,25710,0.079,187,0.0,NaN,yuyuzhen3194,0.007,yuyuzhen3194:0.72%,fengying9449:0.68%,pengfan5076:0.67%,linbin8358:0.67%,lijing7913:0.66%,likun8302:0.33%,maohaiyan4824:0.33%,chengli6873:0.32%,yanglin6562:0.32%,chenying2872:0.32%
2,department,object,25710,0.079,5,0.0,NaN,rd,0.656,rd:65.60%,sales:17.08%,other:4.33%,accounting:3.08%,hr:1.99%,rd:65.60%,sales:17.08%,other:4.33%,accounting:3.08%,hr:1.99%
3,ip_transform,object,25710,0.000,1192,0.0,NaN,H0TKapkPL,0.008,H0TKapkPL:0.76%,YBCE8ld50:0.70%,2qWPkWg5V:0.70%,8J3dZCu0A:0.69%,4hPUiX1CK:0.68%,88aHOvoHa:0.00%,358EfARvQ:0.00%,4L9vshrCi:0.00%,9RzuSrhOL:0.00%,LWv4Mjkys:0.00%
4,device_num_transform,object,25710,0.000,505,0.0,NaN,K8Ith9mjHsKo,0.008,K8Ith9mjHsKo:0.79%,3wDqyLqvVCn1:0.72%,4BWxjoSreaOm:0.71%,uRYWimJ18UEk:0.70%,sdN7y26qL30M:0.70%,E9qxuBiAo3Ju:0.00%,AF9IY4wFm5vY:0.00%,jI1j9ekI0wfW:0.00%,IN0qCiPw2eHv:0.00%,MmIkTEts5OIC:0.00%
5,browser_version,object,25710,0.000,8,0.0,NaN,edge_93,0.367,edge_93:36.69%,chrome_90:33.55%,safari_13:9.37%,chrome_77:5.57%,firefox_78:5.11%,chrome_77:5.57%,firefox_78:5.11%,chrome_93:4.20%,ie_11:3.50%,ie_9:2.02%
6,browser,object,25710,0.000,5,0.0,NaN,chrome,0.433,chrome:43.31%,edge:36.69%,safari:9.37%,ie:5.52%,firefox:5.11%,chrome:43.31%,edge:36.69%,safari:9.37%,ie:5.52%,firefox:5.11%
7,os_type,object,25710,0.000,2,0.0,NaN,win,0.906,win:90.63%,macos:9.37%,None,None,None,None,None,None,win:90.63%,macos:9.37%
8,os_version,object,25710,0.000,4,0.0,NaN,win10,0.765,win10:76.51%,win7:10.50%,macos_big_sur_11:9.37%,win11:3.62%,None,None,win10:76.51%,win7:10.50%,macos_big_sur_11:9.37%,win11:3.62%
9,op_datetime,object,25710,0.000,25542,0.0,NaN,2022-06-23 10:51:17,0.000,2022-06-23 10:51:17:0.01%,2022-06-01 11:38:12:0.01%,2022-05-25 15:09:56:0.01%,2022-05-06 19:34:25:0.01%,2022-06-07 09:10:38:0.01%,2022-05-24 18:01:53:0.00%,2022-05-25 10:29:21:0.00%,2022-05-07 16:09:19:0.00%,2022-06-30 14:19:20:0.00%,2022-06-28 14:50:21:0.00%


## 特征工程

特征衍生

In [9]:
df = pd.concat([df_row_train, df_row_val])
# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 几点钟
df['hour'] = df['op_datetime'].dt.hour
# 周几
df['dayofweek'] = df['op_datetime'].dt.dayofweek
# 一个月的第几天
df['day'] = df['op_datetime'].dt.day
# 一年的第几月
df['month'] = df['op_datetime'].dt.month


# 用户名-认证时间
df = df.sort_values(by=['user_name', 'op_datetime']).reset_index(drop=True)
# datetime转int时间戳
df['ts'] = df['op_datetime'].values.astype(np.int64) // 10 ** 9
# 按客户进行统计这次认证和上次认证的时间差
df['ts1'] = df.groupby('user_name')['ts'].shift(1)
# 按客户进行统计这次认证和上上次认证的时间差
df['ts2'] = df.groupby('user_name')['ts'].shift(2)
# 按客户进行统计这次认证和上上上次认证的时间差
df['ts3'] = df.groupby('user_name')['ts'].shift(3)
df['ts_diff1'] = df['ts1'] - df['ts']
df['ts_diff2'] = df['ts2'] - df['ts']
df['ts_diff3'] = df['ts3'] - df['ts']

df['hour_sin'] = np.sin(df['hour']/24*2*np.pi)
df['hour_cos'] = np.cos(df['hour']/24*2*np.pi)

In [10]:
cat_f = ['user_name', 'department', 'ip_transform', 'device_num_transform', 'browser_version', 'browser',
          'os_type', 'os_version', 'ip_type', 'op_city', 'log_system_transform', 'url',]
for f in cat_f:
    le = LabelEncoder()
    df[f] = le.fit_transform(df[f])
    df[f+'_ts_diff_mean'] = df.groupby([f])['ts_diff1'].transform('mean')
    df[f+'_ts_diff_std'] = df.groupby([f])['ts_diff1'].transform('std')
    df[f+'_ts_diff2_mean'] = df.groupby([f])['ts_diff2'].transform('mean')
    df[f+'_ts_diff2_std'] = df.groupby([f])['ts_diff2'].transform('std')
    df[f+'_ts_diff3_mean'] = df.groupby([f])['ts_diff3'].transform('mean')
    df[f+'_ts_diff3_std'] = df.groupby([f])['ts_diff3'].transform('std')

df_train = df[df[y_label].notna()].reset_index(drop=True)
df_test = df[df[y_label].isna()].reset_index(drop=True)

In [11]:
feats = [f for f in df_test if f not in [y_label, 'id',
                                         'op_datetime', 'op_month', 'ts', 'ts1', 'ts2']]
print(feats)
print(df_train[feats].shape, df_test[feats].shape)

['user_name', 'department', 'ip_transform', 'device_num_transform', 'browser_version', 'browser', 'os_type', 'os_version', 'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url', 'hour', 'dayofweek', 'day', 'month', 'ts3', 'ts_diff1', 'ts_diff2', 'ts_diff3', 'hour_sin', 'hour_cos', 'user_name_ts_diff_mean', 'user_name_ts_diff_std', 'user_name_ts_diff2_mean', 'user_name_ts_diff2_std', 'user_name_ts_diff3_mean', 'user_name_ts_diff3_std', 'department_ts_diff_mean', 'department_ts_diff_std', 'department_ts_diff2_mean', 'department_ts_diff2_std', 'department_ts_diff3_mean', 'department_ts_diff3_std', 'ip_transform_ts_diff_mean', 'ip_transform_ts_diff_std', 'ip_transform_ts_diff2_mean', 'ip_transform_ts_diff2_std', 'ip_transform_ts_diff3_mean', 'ip_transform_ts_diff3_std', 'device_num_transform_ts_diff_mean', 'device_num_transform_ts_diff_std', 'device_num_transform_ts_diff2_mean', 'device_num_transform_ts_diff2_std', 'device_num_transform_ts_diff3_mean', 'device_num_transfo

## modeling

In [12]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,

    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [13]:
fold_num = 5
seeds = [2222]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label])
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label])
        model = lgb.train(params, train, valid_sets=[val], )
                          #num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.926684
[2]	valid_0's auc: 0.929117
[3]	valid_0's auc: 0.929994
[4]	valid_0's auc: 0.930775
[5]	valid_0's auc: 0.931052
[6]	valid_0's auc: 0.930931
[7]	valid_0's auc: 0.930884
[8]	valid_0's auc: 0.930688
[9]	valid_0's auc: 0.930617
[10]	valid_0's auc: 0.930733
[11]	valid_0's auc: 0.930807
[12]	valid_0's auc: 0.931011
[13]	valid_0's auc: 0.931406
[14]	valid_0's auc: 0.932119
[15]	valid_0's auc: 0.932138
[16]	valid_0's auc: 0.932467
[17]	valid_0's auc: 0.932313
[18]	valid_0's auc: 0.932723
[19]	valid_0's auc: 0.933637
[20]	valid_0's auc: 0.933346
[21]	valid_0's auc: 0.933404
[22]	valid_0's auc: 0.933239
[23]	valid_0's auc: 0.933301
[24]	valid_0's auc: 0.933262
[25]	valid_0's auc: 0.933069
[26]	valid_0's auc: 0.932936
[27]	valid_0's auc: 0.93294
[28]	valid_0's auc: 0.933233
[29]	valid_0's auc: 0.933259
[30]	valid_0's auc: 0.933421
[31]	valid_0's auc: 0.933564
[32]	valid_0's auc: 0.933533
[33]	valid_0's auc: 0.933876
[34]	valid_0's auc: 0.934486
[35]	valid

----------- 3
[1]	valid_0's auc: 0.921447
[2]	valid_0's auc: 0.929447
[3]	valid_0's auc: 0.926632
[4]	valid_0's auc: 0.927748
[5]	valid_0's auc: 0.929947
[6]	valid_0's auc: 0.930134
[7]	valid_0's auc: 0.930595
[8]	valid_0's auc: 0.930519
[9]	valid_0's auc: 0.930511
[10]	valid_0's auc: 0.930493
[11]	valid_0's auc: 0.931101
[12]	valid_0's auc: 0.931077
[13]	valid_0's auc: 0.932119
[14]	valid_0's auc: 0.932543
[15]	valid_0's auc: 0.932548
[16]	valid_0's auc: 0.932233
[17]	valid_0's auc: 0.931885
[18]	valid_0's auc: 0.93166
[19]	valid_0's auc: 0.931813
[20]	valid_0's auc: 0.931841
[21]	valid_0's auc: 0.931958
[22]	valid_0's auc: 0.932475
[23]	valid_0's auc: 0.932579
[24]	valid_0's auc: 0.932456
[25]	valid_0's auc: 0.932178
[26]	valid_0's auc: 0.933137
[27]	valid_0's auc: 0.933322
[28]	valid_0's auc: 0.93336
[29]	valid_0's auc: 0.933633
[30]	valid_0's auc: 0.933935
[31]	valid_0's auc: 0.93422
[32]	valid_0's auc: 0.934148
[33]	valid_0's auc: 0.934661
[34]	valid_0's auc: 0.934487
[35]	valid_0

,name,importance
9,http_status_code,39508.476062
22,hour_cos,38747.283996
35,ip_transform_ts_diff_mean,32579.986290
13,hour,23962.044299
20,ts_diff3,12645.399376
36,ip_transform_ts_diff_std,8546.865213
15,day,8526.957920
14,dayofweek,8111.978896
18,ts_diff1,6867.216748
17,ts3,6839.330259


0.9340896608734036

0.001295563660236228